# NHL Data Exploration Notebook

This notebook provides a simple interface to explore and analyze NHL data stored in your MySQL database. You can modify queries and visualizations to explore different aspects of the data.

## Overview
- Connect to your MySQL database
- Explore database structure
- Run custom queries
- Visualize data trends
- Perform basic statistical analysis

## 1. Import Required Libraries

Let's start by importing all the libraries we'll need for data exploration and visualization.

In [1]:
# Import required libraries
import mysql.connector
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from config import db_config
from plotly.subplots import make_subplots
import warnings
import ipywidgets as widgets
from IPython.display import display
from dash import Dash, html, dcc, Input, Output
import dash_bootstrap_components as dbc


# Configure display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
warnings.filterwarnings('ignore')

# Set plotting style (using default instead of seaborn-v0_8 to avoid crash)
plt.style.use('default')
sns.set_palette("husl")

print("Libraries imported successfully!")

Libraries imported successfully!


## 2. Connect to MySQL Database

Update the connection parameters below with your database credentials.

In [2]:
# Establish connection
connection = mysql.connector.connect(
    host=db_config["host"],
    user=db_config["user"],
    password=db_config["password"],
    database=db_config["database"]
)
cursor = connection.cursor()

In [3]:
teams_dict = {
    'team_id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 46, 47, 48, 49, 52, 53, 54, 55, 56, 59, 68],
    'team_name': ['New Jersey Devils', 'New York Islanders', 'New York Rangers', 'Philadelphia Flyers', 'Pittsburgh Penguins', 'Boston Bruins', 'Buffalo Sabres', 'Montreal Canadiens', 'Ottawa Senators', 'Toronto Maple Leafs', 'Atlanta Thrashers', 'Carolina Hurricanes', 'Florida Panthers', 'Tampa Bay Lightning', 'Washington Capitals', 'Chicago Blackhawks', 'Detroit Red Wings', 'Nashville Predators', 'St Louis Blues', 'Calgary Flames', 'Colorado Avalanche', 'Edmonton Oilers', 'Vancouver Canucks', 'Anaheim Ducks', 'Dallas Stars', 'Los Angeles Kings', 'Pheonix Coyotes', 'San Jose Sharks', 'Columbus Blue Jackets', 'Minnesota Wild', 'Minnesota North Stars', 'Quebec Nordique', 'Winnipeg Jets (1979)', 'Hartford Whalers', 'Colorado Rockies', 'Oakland Seals', 'Atlanta Flames', 'Kansas City Scouts', 'Cleveland Barons', 'Winnipeg Jets', 'Arizona Coyotes', 'Vegas Golden Knights', 'Seattle Kraken', 'California Golden Seals', 'Utah Hockey Club', 'Utah Mammoth'],
    'team_abbreviation': ['NJD', 'NYI', 'NYR', 'PHI', 'PIT', 'BOS', 'BUF', 'MTL', 'OTT', 'TOR', 'ATL', 'CAR', 'FLA', 'TBL', 'WSH', 'CHI', 'DET', 'NSH', 'STL', 'CGY', 'COL', 'EDM', 'VAN', 'ANA', 'DAL', 'LAK', 'PHX', 'SJS', 'CBJ', 'MIN', 'MNS', 'QUE', 'WIN', 'HFD', 'CLR', 'OAK', 'AFM', 'KCS', 'CLE', 'WPG', 'ARI', 'VGK', 'SEA', 'CGS', 'UTA', 'UTA']
}

## 3. Explore Database Structure

Let's see what tables and data we have available in the database.

In [7]:
# Show all tables in the database
cursor.execute("SHOW TABLES")
tables = cursor.fetchall()

print("Available tables:")
table_names = []
for table in tables:
    table_name = table[0]
    table_names.append(table_name)
    print(f"- {table_name}")

print(f"\nTotal tables: {len(table_names)}")

# Function to easily query database and return DataFrame
def query_to_df(query, params=None):
    """Execute query and return results as pandas DataFrame"""
    try:
        if params:
            cursor.execute(query, params)
        else:
            cursor.execute(query)
        
        # Get column names
        columns = [desc[0] for desc in cursor.description]
        
        # Fetch data and create DataFrame
        data = cursor.fetchall()
        df = pd.DataFrame(data, columns=columns)
        return df
    except Exception as e:
        print(f"Query error: {e}")
        return pd.DataFrame()

Available tables:
- events
- games
- players
- players_season
- roster_players
- seasons
- seasons_end_standings
- teams

Total tables: 8


In [8]:
# Explore table structures (modify table name as needed)
def explore_table_structure(table_name):
    """Show structure and sample data for a table"""
    print(f"\n=== Table: {table_name} ===")
    
    # Show table structure
    cursor.execute(f"DESCRIBE {table_name}")
    structure = cursor.fetchall()
    
    print("Columns:")
    for col in structure:
        print(f"  {col[0]} ({col[1]}) - {col[2]} - {col[3]}")
    
    # Show row count
    cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
    row_count = cursor.fetchone()[0]
    print(f"\nTotal rows: {row_count:,}")
    
    # Show sample data
    sample_df = query_to_df(f"SELECT * FROM {table_name} LIMIT 5")
    print(f"\nSample data:")
    return sample_df

# Explore the first table (modify index or table name as needed)
if table_names:
    first_table = table_names[0]  # Change index to explore different tables
    sample_data = explore_table_structure(first_table)
    display(sample_data)


=== Table: events ===
Columns:
  event_id (int) - NO - PRI
  game_id (bigint) - NO - PRI
  period_number (int) - YES - 
  period_type (varchar(10)) - YES - 
  time_in_period (varchar(10)) - YES - 
  time_remaining (varchar(10)) - YES - 
  situation_code (varchar(10)) - YES - 
  home_team_defending_side (varchar(10)) - YES - 
  type_code (int) - YES - 
  type_desc_key (varchar(50)) - YES - 
  sort_order (int) - YES - 
  x_coord (int) - YES - 
  y_coord (int) - YES - 
  zone_code (varchar(5)) - YES - 
  shot_type (varchar(20)) - YES - 
  blocking_Player_id (int) - YES - 
  shooting_player_id (int) - YES - 
  goalie_in_net_id (int) - YES - 
  player_id (int) - YES - 
  event_owner_team_id (int) - YES - 
  away_sog (int) - YES - 
  home_sog (int) - YES - 
  hitting_player_id (int) - YES - 
  hittee_player_id (int) - YES - 
  reason (varchar(255)) - YES - 
  secondary_reason (varchar(255)) - YES - 
  losing_player_id (int) - YES - 
  winning_player_id (int) - YES - 

Total rows: 8,424,272


,event_id,game_id,period_number,period_type,time_in_period,time_remaining,situation_code,home_team_defending_side,type_code,type_desc_key,sort_order,x_coord,y_coord,zone_code,shot_type,blocking_Player_id,shooting_player_id,goalie_in_net_id,player_id,event_owner_team_id,away_sog,home_sog,hitting_player_id,hittee_player_id,reason,secondary_reason,losing_player_id,winning_player_id
0,2,2009020257,1,REG,00:16,19:44,1551,,504,giveaway,8,89,-32,D,,-1,-1,-1,8469623,30,-1,-1,-1,-1,,,-1,-1
1,2,2009020457,1,REG,00:00,20:00,-1,,520,period-start,5,-1,-1,,,-1,-1,-1,-1,-1,-1,-1,-1,-1,,,-1,-1
2,2,2009020482,1,REG,00:22,19:38,1551,,503,hit,9,92,27,D,,-1,-1,-1,-1,2,-1,-1,8468309,8462041,,,-1,-1
3,2,2009020709,1,REG,01:56,18:04,1551,,507,missed-shot,20,-51,-25,O,wrist,-1,8471657,8455710,-1,21,-1,-1,-1,-1,wide-of-net,,-1,-1
4,2,2010020090,1,REG,00:00,20:00,-1,,520,period-start,5,-1,-1,,,-1,-1,-1,-1,-1,-1,-1,-1,-1,,,-1,-1


## 4. Data Visualization

Create some quick visualizations to explore patterns in your data.

In [ ]:
cursor.execute("""
    SELECT *
    FROM seasons_end_standings
""")

seasons_end_standings_df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

cursor.execute("""
    SELECT *
    FROM teams
""")

teams = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

#Create an interactable table giving a season overview for the league with an option to choose which season

#adjust dataframe

# Rename columns to more user-friendly names
seasons_end_standings_df.rename(columns={
    'season_id': 'Season',
    'ot_losses': 'Overtime Losses',
    'games_played': 'Games Played',
    'points': 'Points',
    'wins': 'Wins',
    'losses': 'Losses'
}, inplace=True)

#Season values 12345678 -> 1234
seasons_end_standings_df['Season'] = seasons_end_standings_df['Season'].astype(str).str[:4]

available_seasons = seasons_end_standings_df['Season'].unique()

#Team -> get team_name from the teams df for each Team(formerly team_id)
# Merge the seasons_end_standings_df with the teams DataFrame to get team_name
seasons_end_standings_df = seasons_end_standings_df.merge(
    teams[['team_id', 'team_name']],  # Select only the relevant columns from teams
    left_on='team_id',                  # Match on the Team column in seasons_end_standings_df
    right_on='team_id',              # Match on the team_id column in teams
    how='left'                       # Use a left join to keep all rows from seasons_end_standings_df
)

# Drop the now redundant 'team_id' column
seasons_end_standings_df.drop(columns=['team_id'], inplace=True)

# Rename 'team_name' to 'Team' for clarity
seasons_end_standings_df.rename(columns={'team_name': 'Team'}, inplace=True)

columns = ['Team'] + [col for col in seasons_end_standings_df.columns if col != 'Team']
seasons_end_standings_df = seasons_end_standings_df[columns]

# Sort the DataFrame by the 'points' column in descending order
seasons_end_standings_df = seasons_end_standings_df.sort_values(by='Points', ascending=False)

# Select and reorder specific fields
selected_columns = ['Team', 'Season', 'Wins', 'Losses', 'Overtime Losses', 'Points', 'Games Played']
seasons_end_standings_df = seasons_end_standings_df[selected_columns]

# Create a dropdown widget for selecting a season
season_selector = widgets.Dropdown(
    options=available_seasons,
    description='Season:',
    value=available_seasons[0],
    style={'description_width': 'initial'}
)

# Function to update and display the table based on the selected season
def update_table(selected_season):
    filtered_df = seasons_end_standings_df[
        (seasons_end_standings_df['Season'] == selected_season) &
        (seasons_end_standings_df['Games Played'] > 0)
    ]
    display(filtered_df)  # Display the filtered DataFrame as a table

# Use the interact function to link the dropdown to the update_table function
widgets.interact(update_table, selected_season=season_selector)

interactive(children=(Dropdown(description='Season:', options=('1942', '1943', '1944', '1945', '1946', '1947',…

<function __main__.update_table(selected_season)>

In [ ]:
cursor.execute('''
    SELECT * FROM games
    WHERE game_id = 2024010008
''')

cursor.fetchall()

cursor.execute('''
    SELECT * FROM events
    WHERE game_id = 2024010008
''')

events_headers = [desc[0] for desc in cursor.description]  # This gives you the column names
events = cursor.fetchall()

In [ ]:
events_df = pd.DataFrame(events, columns=events_headers)
events_df.head()

,event_id,game_id,period_number,period_type,time_in_period,time_remaining,situation_code,home_team_defending_side,type_code,type_desc_key,sort_order,x_coord,y_coord,zone_code,shot_type,blocking_Player_id,shooting_player_id,goalie_in_net_id,player_id,event_owner_team_id,away_sog,home_sog,hitting_player_id,hittee_player_id,reason,secondary_reason,losing_player_id,winning_player_id
0,8,2024010008,1,REG,01:12,18:48,1551,left,516,stoppage,27,-1,-1,,,-1,-1,-1,-1,-1,-1,-1,-1,-1,offside,,-1,-1
1,9,2024010008,1,REG,02:59,17:01,1551,left,516,stoppage,45,-1,-1,,,-1,-1,-1,-1,-1,-1,-1,-1,-1,puck-in-netting,,-1,-1
2,10,2024010008,1,REG,05:27,14:33,1551,left,516,stoppage,73,-1,-1,,,-1,-1,-1,-1,-1,-1,-1,-1,-1,goalie-stopped-after,,-1,-1
3,11,2024010008,1,REG,06:19,13:41,1551,left,516,stoppage,82,-1,-1,,,-1,-1,-1,-1,-1,-1,-1,-1,-1,skater-puck-frozen,tv-timeout,-1,-1
4,12,2024010008,1,REG,06:36,13:24,1551,left,516,stoppage,88,-1,-1,,,-1,-1,-1,-1,-1,-1,-1,-1,-1,goalie-stopped-after,,-1,-1


In [ ]:
import plotly.express as px

# Filter out rows with missing zone_code if needed
filtered_df = events_df[events_df['zone_code'].notnull()]
filtered_df = filtered_df[~filtered_df['type_desc_key'].isin(['stoppage', 'faceoff'])]

# Create a descriptive hover text for each event
filtered_df['hover_text'] = (
    "Type: " + filtered_df['type_desc_key'] +
    "<br>Period: " + filtered_df['period_number'].astype(str) +
    "<br>Time: " + filtered_df['time_in_period'].astype(str)
    # Add more fields as needed
)

# Sort by period_number (ascending) and time_in_period (ascending)
filtered_df = filtered_df.sort_values(['period_number', 'time_in_period'])

# Create the event_time column after sorting
filtered_df['event_time'] = (
    filtered_df['period_number'].astype(str) + '-' + filtered_df['time_in_period'].astype(str)
)

# (Plotting code remains the same)
def time_str_to_minute(time_str):
    try:
        minutes, seconds = map(int, time_str.split(':'))
        return minutes  # Use minutes only for frame
    except Exception:
        return 0  # Fallback for malformed strings

filtered_df['minute'] = filtered_df['time_in_period'].apply(time_str_to_minute)
filtered_df['frame'] = filtered_df['period_number'].astype(str) + '-' + filtered_df['minute'].astype(str)
fig = px.scatter(
    filtered_df,
    x='x_coord',
    y='y_coord',
    color='type_desc_key',
    hover_name='type_desc_key',
    hover_data={'hover_text': True, 'x_coord': False, 'y_coord': False},
    labels={'x_coord': 'X Coordinate', 'y_coord': 'Y Coordinate'},
    title='Event Locations for Game 2024010008',
    animation_frame='frame',
    animation_group='type_desc_key'
)

fig.update_traces(
    hovertemplate='%{customdata[0]}<extra></extra>',
    customdata=filtered_df[['hover_text']]
)

fig.update_layout(
    legend_title_text='Event Type',
    width=900,
    height=600
)

fig.show()

In [4]:
cursor.execute('''
    SELECT * FROM games
    WHERE game_id = 2024010008
''')

c = cursor.fetchall()

In [6]:
#get structure of sql database and tables and show details



# Show details for each table
for table in tables:
    cursor.execute(f"DESCRIBE {table[0]}")
    columns = cursor.fetchall()
    print(f"Details for table {table[0]}:")
    for column in columns:
        print(f"  - {column[0]}: {column[1]}")
    print()

Details for table events:
  - event_id: int
  - game_id: bigint
  - period_number: int
  - period_type: varchar(10)
  - time_in_period: varchar(10)
  - time_remaining: varchar(10)
  - situation_code: varchar(10)
  - home_team_defending_side: varchar(10)
  - type_code: int
  - type_desc_key: varchar(50)
  - sort_order: int
  - x_coord: int
  - y_coord: int
  - zone_code: varchar(5)
  - shot_type: varchar(20)
  - blocking_Player_id: int
  - shooting_player_id: int
  - goalie_in_net_id: int
  - player_id: int
  - event_owner_team_id: int
  - away_sog: int
  - home_sog: int
  - hitting_player_id: int
  - hittee_player_id: int
  - reason: varchar(255)
  - secondary_reason: varchar(255)
  - losing_player_id: int
  - winning_player_id: int

Details for table games:
  - game_id: int
  - season_id: int
  - game_type: int
  - date: date
  - home_team_id: int
  - away_team_id: int
  - home_score: int
  - away_score: int
  - game_outcome: varchar(255)
  - winning_goalie_id: int
  - winning_goal_sc

## 5. Experimental Section

Use this section to write and test your own custom queries and analysis. This is your playground to tinker with the data!

In [ ]:
# Your experimental code goes here!
# Try different queries, create custom visualizations, or perform statistical analysis

# Example: Custom query
# my_query = """
# SELECT your_columns
# FROM your_table
# WHERE your_conditions
# """
# my_data = query_to_df(my_query)
# display(my_data)

# Example: Custom visualization
# your_viz_code_here

print("Ready for experimentation! Write your custom code above.")

## 6. Clean Up

Don't forget to close your database connection when you're done!

In [ ]:
# Close database connection
try:
    if cursor:
        cursor.close()
    if connection:
        connection.close()
    print("Database connection closed successfully")
except:
    print("No active connection to close")